In [ ]:
import os
import pandas as pd
import numpy as np
import sys
import win32com.client
import getpass
import datetime
import pywintypes
import matplotlib.pyplot as plt
import pickle
%matplotlib inline
#ann model
import tensorflow as tf
from tensorflow.keras import layers
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler


import matplotlib.ticker as mtick
import math
from sklearn.ensemble import IsolationForest
pd.set_option('display.width', None)
pd.set_option('display.max_column',None)
pd.set_option('display.max_rows',None)


<h2>ANN</h2>

In [ ]:
def read_data():
    """
    Returns all the data that needs to be used for ANN.
    Output(3 dataframes): clinical data, clinical data (OHE), bills grouped by time period
    """
    master = pd.read_pickle("C:\\SMU_v2\\price_timeperiod.pkl")
    
    listToKeep = ['NRIC','dx_date','tstage','nstage', 'Mstage', 'ER', 'PR',\
               'Her2', 'size_precise', 'nodespos', 'Age_@_Dx']
    
    clinical = master[listToKeep]
    
    OHE = [i for i in clinical.columns if not (i in  ["NRIC", 'Age_@_Dx', 'size_precise', 'nodespos','dx_date'])]
    x_clinical = pd.get_dummies(clinical,columns=OHE,dummy_na=True).reset_index(drop=True)
    prices_grouped = master[["NRIC","before_6m", "after_6m", "after_1y", "after_2y", "after_3y", "after_4y",
               "after_5y", "after_6y", "after_7y","after_8y", "after_9y", "after_10y"]]
    return clinical.reset_index(drop=True), x_clinical.reset_index(drop=True), prices_grouped.reset_index(drop=True)

def scale_data(data,scale_obj):
    """
    transforms then scales data according to min-max
    """
#     data = data.apply(np.log1p)
    prices_grouped_scaled = pd.DataFrame(scale_obj.fit_transform(data))
    return prices_grouped_scaled

def scale_data_reverse(data,scale_obj):
    """
    returns a dataframe that reverses the min-max that was done previously
    """
    data = pd.DataFrame(scale_obj.inverse_transform(data))
#     predictions_scaled_reverse = data.apply(np.expm1)
    return data

def ann_structure(input_shape,output_units):
    """
    function to declare ANN structure. just for code cleaniness
    """
    model = tf.keras.Sequential()
    model.add(layers.Dense(32, input_shape=(input_shape,)))         # input layer
    model.add(layers.Dense(32, activation=tf.nn.leaky_relu))      # one hidden layer
    model.add(layers.Dropout(.5))
    model.add(layers.Dense(32, activation=tf.nn.leaky_relu))      # one hidden layer
    
    model.add(layers.Dense(32, activation=tf.nn.leaky_relu))      # one hidden layer
    model.add(layers.Dense(32, activation=tf.nn.leaky_relu))      # one hidden layer
    model.add(layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, 
                                        beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', 
                                        moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, 
                                        beta_constraint=None, gamma_constraint=None)
) 
    model.add(layers.Dense(32, activation=tf.nn.leaky_relu))      # one hidden layer
    model.add(layers.Dense(32, activation=tf.nn.leaky_relu))      # one hidden layer
    model.add(layers.Dense(32, activation=tf.nn.leaky_relu))      # one hidden layer
    model.add(layers.Dense(32, activation=tf.nn.leaky_relu))      # one hidden layer
    model.add(layers.Dense(16, activation=tf.nn.leaky_relu))      # one hidden layer
    model.add(layers.Dropout(.5))
    model.add(layers.Dense(16, activation=tf.nn.leaky_relu))      # one hidden layer
    model.add(layers.Dense(16, activation=tf.nn.leaky_relu))      # one hidden layer
    model.add(layers.Dense(16, activation=tf.nn.leaky_relu))      # one hidden layer
    model.add(layers.Dense(16, activation=tf.nn.leaky_relu))      # one hidden layer
    model.add(layers.Dropout(.5))
    model.add(layers.Dense(16, activation=tf.nn.leaky_relu))      # one hidden layer
    model.add(layers.Dense(16, activation=tf.nn.leaky_relu))      # one hidden layer
    model.add(layers.Dense(output_units, activation=tf.nn.leaky_relu))   # one output layer with 1 outputs
    return model

def remove_out_of_range(data):
    """
    determines index of data where there is no additional information
    """
    y1 = data[data["after_1y"].isnull()].index
    y2 = data[data["after_2y"].isnull()].index
    y5 = data[data["after_5y"].isnull()].index
    y10 = data[data["after_10y"].isnull()].index
    return {
        "y1":[4,y1], 
        "y2":[5,y2], 
        "y5":[8,y5], 
        "y10":[13,y10]}

def remove_meaningless_data(data):
    """
    returns index of all rows that do not add any additional input. aka all fields are 0
    """
    return data[data.sum(axis=1)==0].index

def drop_by_index(X,y,indexes):
    """
    helper function to drop rows of dataframe and return new dataframe without those rows with indexes resetted
    """
    y = y.drop(indexes)
    X = X.drop(indexes)
    X = X.reset_index(drop=True)
    y = y.reset_index(drop=True)
    return(X,y)

def scheduler(epoch):
    """
    to reduce learning rate as epoch number increases
    """
    if epoch < 30:
        return 0.001
    else:
        return 0.001 * math.exp(0.1 * (10 - int(epoch)))
    
def process_time_period(data,scope):
    """
    Takes in yearly healthcare costs of patients and processes it into 1,2,5,10 year values
    """
    y = pd.DataFrame()
    y["6 months before"] = data.iloc[:,0]
    y["6 months after"] = data.iloc[:,1]
    y["1 year after"] = data.iloc[:,2]
    if scope != "y1":
        y["2 years after"] = data.iloc[:,3]
        if scope != "y2":
            y["5 years after"] = data.iloc[:,4:7].sum(axis=1)
            if scope != "y5":
                y["10 years after"] = data.iloc[:,7:].sum(axis=1)
    return y
    
def make_prediction(all_users,user,model,mms):
    """
    Given user data(dataframe) and the trained model, outputs the predicted values.
    
    Only works if all items in new user data has appeared at least once before
    """
    all_users = all_users.reset_index().drop(columns="index")
    last_row = all_users.shape[0]
    all_users = all_users.append(user)
    all_users = all_users.drop(columns=["NRIC","dx_date"])
    OHE = [i for i in all_users.columns if not (i in  ["NRIC", 'Age_@_Dx', 'size_precise', 'nodespos','dx_date'])]
    usersOHE = pd.get_dummies(all_users,columns=OHE,dummy_na=True).reset_index().drop(columns="index")
    prediction_x = usersOHE
#     print(prediction_x.shape)
    pred = model.predict(prediction_x)
    predictions_scaled_reverse = pd.DataFrame(mms.inverse_transform(pred),columns=["6 months before","6 months after",
                                                                                   "1 year after","2 years after",
                                                                                   "5 years after","10 years after"][:pred.shape[1]])
    return pd.DataFrame([pd.DataFrame(predictions_scaled_reverse).iloc[last_row]]).reset_index().drop(columns="index")

def make_comparison(all_users,all_users_OHE,NRIC,bills,model,mms):
    """
    Given a specific user, calculate out his actual cost and predicted costs
    """
    x = all_users[all_users["NRIC"] == NRIC]
    pred = make_prediction(all_users,x,model,mms)
    pred["Status"] = "Prediction"
    if pred.shape[1] == 7:
        limit = 0
    elif pred.shape[1] == 6:
        limit = 1
    else:
        limit = 2
    y_test = process_time_period(bills[all_users["NRIC"] == NRIC],limit)
    y_test["Status"] = "True data"
    y_test.columns = ["6 months before","6 months after","1 year after",
                      "2 years after","5 years after","10 years after"][:pred.shape[1]-1] +["Status"]
    
    to_return = y_test.append(pred)
    return to_return

def get_percentage(df1,df2,percentage):
    """
    Given 2 dataframes, get the difference between the dataframes, 
    and return number of records that fall within a given percentage.
    Eg: df1 contains 5 values [1,2,3,4,5]. Df2 contains [1,2,9,4,5]. 4 out of 5 values in 
    df1 fall within +- 5(percentage)% of the values in the same postion in df2. 
    Thus function will return 4/5 or 0.8
    """
    process = lambda s1,s2: abs(s1-s2)/s2 < percentage 
    combined = df1.combine(df2, process)
    total_count = (df1.shape[0] * df1.shape[1])
    minus = sum([pd.value_counts(df2[i].values)[0] for i in df2.columns])
    total_count -= minus
    return combined.sum().sum() / total_count

def display_graph(scope,predictions_scaled_reverse,y_test_scaled_reverse):
    graph = pd.DataFrame(np.arange(0,2,.01),columns=["Percentage"])
    graph["viz"] = graph.applymap(lambda x: get_percentage(predictions_scaled_reverse,y_test_scaled_reverse,x))
    show = graph.plot.area(x="Percentage")
    show.set_title("Model performance ({})".format(scope))
    show.set_xlabel("Percentage Difference from Ground Truth")
    show.set_ylabel("Percentage of all our predictions")
    show.xaxis.set_major_formatter(mtick.PercentFormatter(1.0))
    show.yaxis.set_major_formatter(mtick.PercentFormatter(1.0))
    
def get_data(scope,clinicalOHE, bills_grouped, outlier=False):
    index = remove_indexes[scope]

    X = clinicalOHE
    y = bills_grouped.iloc[:,:index[0]]  

    X = X.iloc[:,2:]
    y = process_time_period(y.iloc[:,1:],scope)

    print("Data shape original: {}".format(X.shape[0]))
    X,y = drop_by_index(X,y,index[1])

    print("Data shape removing data out of scope: {}".format(X.shape[0]))


    meaningless = remove_meaningless_data(y)
    X,y = drop_by_index(X,y,meaningless)

    print("Data shape meaningless data: {}".format(X.shape[0]))



    if outlier:
        clf = IsolationForest(contamination="auto",behaviour="new",random_state=42)
        out = clf.fit_predict(y)
        out_df = pd.DataFrame(out,columns=["outlier"])
        remove = out_df[out_df["outlier"] ==-1].index
        X,y = drop_by_index(X,y,remove)

        print("Data shape after removing outliers: {}".format(X.shape[0]))

    mms = MinMaxScaler()
    y_scaled = scale_data(y,mms)
    return X,y_scaled,mms



In [ ]:
clinical, clinicalOHE, bills_grouped = read_data()

remove_indexes = remove_out_of_range(bills_grouped)


In [ ]:
#10 years
scope = "y10"
X,y,mms = get_data(scope,clinicalOHE,bills_grouped)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = ann_structure(X.shape[1],y.shape[1])
model.compile(optimizer=tf.keras.optimizers.Adam(), 
          loss='mean_squared_error')
# Run the stochastic gradient descent for specified epochs
epochs = 100
filepath="weights.best.{}.h5".format(scope)
callbacks_list = []
callbacks_list.append(ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=True))
# callbacks_list.append(LearningRateScheduler(scheduler))

model.fit(X_train, y_train, epochs=epochs, callbacks = callbacks_list, validation_data=(X_test, y_test))

In [ ]:

pred = pd.DataFrame(model.predict(X_test))
predictions_scaled_reverse = scale_data_reverse(pred,mms)
y_test_scaled_reverse = scale_data_reverse(y_test,mms)
display_graph(scope,predictions_scaled_reverse,y_test_scaled_reverse)

In [ ]:
#5 years
outlier = False
scope = "y5"
X,y,mms = get_data(scope,clinicalOHE,bills_grouped)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = ann_structure(X.shape[1],y.shape[1])
model.compile(optimizer=tf.keras.optimizers.Adam(), 
          loss='mean_squared_error')
# Run the stochastic gradient descent for specified epochs
epochs = 100
filepath="weights.best.{}.h5".format(scope)
callbacks_list = []
callbacks_list.append(ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=True))
# callbacks_list.append(LearningRateScheduler(scheduler))

model.fit(X_train, y_train, epochs=epochs, callbacks = callbacks_list, validation_data=(X_test, y_test))

In [ ]:

pred = pd.DataFrame(model.predict(X_test))
predictions_scaled_reverse = scale_data_reverse(pred,mms)
y_test_scaled_reverse = scale_data_reverse(y_test,mms)
display_graph(scope,predictions_scaled_reverse,y_test_scaled_reverse)

In [ ]:
#2 years
outlier = False
scope = "y2"
X,y,mms = get_data(scope,clinicalOHE,bills_grouped)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = ann_structure(X.shape[1],y.shape[1])
model.compile(optimizer=tf.keras.optimizers.Adam(), 
          loss='mean_squared_error')
# Run the stochastic gradient descent for specified epochs
epochs = 50
filepath="weights.best.{}.h5".format(scope)
callbacks_list = []
callbacks_list.append(ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=True))
# callbacks_list.append(LearningRateScheduler(scheduler))

model.fit(X_train, y_train, epochs=epochs, callbacks = callbacks_list, validation_data=(X_test, y_test))

In [ ]:

pred = pd.DataFrame(model.predict(X_test))
predictions_scaled_reverse = scale_data_reverse(pred,mms)
y_test_scaled_reverse = scale_data_reverse(y_test,mms)
display_graph(scope,predictions_scaled_reverse,y_test_scaled_reverse)

In [ ]:
#1 year
outlier = False
scope = "y1"
X,y,mms = get_data(scope,clinicalOHE,bills_grouped)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = ann_structure(X.shape[1],y.shape[1])
model.compile(optimizer=tf.keras.optimizers.Adam(), 
          loss='mean_squared_error')
# Run the stochastic gradient descent for specified epochs
epochs = 50
filepath="weights.best.{}.h5".format(scope)
callbacks_list = []
callbacks_list.append(ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=True))
# callbacks_list.append(LearningRateScheduler(scheduler))

model.fit(X_train, y_train, epochs=epochs, callbacks = callbacks_list, validation_data=(X_test, y_test))

In [ ]:

pred = pd.DataFrame(model.predict(X_test))
predictions_scaled_reverse = scale_data_reverse(pred,mms)
y_test_scaled_reverse = scale_data_reverse(y_test,mms)
display_graph(scope,predictions_scaled_reverse,y_test_scaled_reverse)